In [1]:
%load_ext line_profiler

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import re
import os

In [3]:
URL = "https://vi.wikipedia.org/w/api.php"
PARAMS = {
    "action": "query",
    "cmlimit": "100",
    "list": "categorymembers",
    "format": "json",
    
    "cmtitle": "",
    "cmtype": ""
}

PARAMS_WIKITEXT = {
    "action": "parse",
    "prop": "wikitext",
    "formatversion": 2,
    "format": "json",
    
    "page": ""
}

SESSION = requests.Session()

In [4]:
def recurseCat(S, catdict, title, level=0, maxlevel=5):
        # RECURSIVE
    PARAMS['cmtitle'] = title
    PARAMS['cmtype'] = "subcat"
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    # print(DATA)
    PAGES = DATA['query']['categorymembers']

    index = 1
    for page in PAGES:
        childtitle = page['title']
        print("- "*level,index, childtitle)
        catdict.append({"cats": childtitle, "parent": title})
        
        if (level < maxlevel):
            recurseCat(S, catdict, childtitle, level+1, maxlevel)
        else:
            print("Max level reached")
        index += 1

In [5]:
import mwparserfromhell

def title_to_infoboxes(title):
    PARAMS_WIKITEXT['page'] = title
    response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
    data = response.json()

    revisions = data['parse']['wikitext']
        
    wikicode = mwparserfromhell.parse(revisions)
    templates = wikicode.filter_templates(matches="Thông tin|Infobox|thông tin")
    
    template_names = list(map(lambda x: str(x.name), templates))
    
    result_temp = ''.join(template_names)
    
    return result_temp

In [6]:
url = 'https://vi.wikipedia.org/w/api.php?action=parse&prop=wikitext&format=json&page='
current = "Nhà Nguyễn"

%lprun -f title_to_infoboxes title_to_infoboxes(current)

Timer unit: 1e-07 s

Total time: 1.86984 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_8960\515650576.py
Function: title_to_infoboxes at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def title_to_infoboxes(title):
     4         1          9.0      9.0      0.0      PARAMS_WIKITEXT['page'] = title
     5         1   15135835.0 15135835.0     80.9      response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
     6         1      13699.0  13699.0      0.1      data = response.json()
     7                                           
     8         1         11.0     11.0      0.0      revisions = data['parse']['wikitext']
     9                                                   
    10         1    2888289.0 2888289.0     15.4      wikicode = mwparserfromhell.parse(revisions)
    11         1     660365.0 660365.0      3.5      templates = wikicode.filter_templates(matches="Thông tin|Infobox|thông tin")
    1

In [7]:
def getAllPages(S, SET_OF_CATS):
    data = []
    
    i = 0
    for cat in SET_OF_CATS:
          # GET ALL PAGES
        PARAMS['cmtitle'] = cat
        PARAMS['cmtype'] = "page"
        DATA = S.get(url=URL, params=PARAMS).json()
        
        PAGES = DATA['query']['categorymembers']
        for page in PAGES:
            page_title = page['title']
            
            if "Bản mẫu:" in page_title: # Skip bản mẫu
                continue;
                
            thispage = {"page": page_title, "cat": cat }
            data.append(thispage)
            print(thispage)
            
        i += 1
        print(f'Progress: {i}', end='\r')
            
    return pd.DataFrame(data)

## TESTING & DEFINITIONS END HERE

In [25]:
toptitle = "Thể loại:Triều đại Việt Nam"
new_cat_dict = [{'cats': toptitle, 'parent': toptitle }]

recurseCat(SESSION, new_cat_dict, toptitle, maxlevel=1)
print("\nThat's all")

 1 Thể loại:Biểu tượng hoàng gia tại Việt Nam
 2 Thể loại:Người Việt Nam theo triều đại
-  1 Thể loại:Nhân vật thời Đinh
Max level reached
-  2 Thể loại:Nhân vật thời Hồ
Max level reached
-  3 Thể loại:Nhân vật thời Hậu Lê
Max level reached
-  4 Thể loại:Nhân vật thời Tiền Lê
Max level reached
-  5 Thể loại:Nhân vật thời Lý
Max level reached
-  6 Thể loại:Nhân vật thời Ngô
Max level reached
-  7 Thể loại:Nhân vật thời Nguyễn
Max level reached
-  8 Thể loại:Nhân vật thời Tây Sơn
Max level reached
-  9 Thể loại:Nhân vật Việt Nam thời kỳ Tiền độc lập
Max level reached
-  10 Thể loại:Nhân vật thời Trần
Max level reached
 3 Thể loại:An Dương Vương
-  1 Thể loại:Âu Lạc
Max level reached
 4 Thể loại:Nhà Đinh
-  1 Thể loại:Công tước nhà Đinh
Max level reached
-  2 Thể loại:Hoàng tộc nhà Đinh
Max level reached
-  3 Thể loại:Lịch sử Việt Nam thời Đinh
Max level reached
-  4 Thể loại:Nhân vật thời Đinh
Max level reached
-  5 Thể loại:Quan lại nhà Đinh
Max level reached
-  6 Thể loại:Vua nhà Đinh


In [27]:
if not os.path.isfile('Categories.csv'):
    df1 = pd.DataFrame(new_cat_dict)
    
    words_place = [
        ':Thời kỳ',
        'Nhà',
        'Triều đại',
    ]

    regex = re.compile('|:'.join(words_place), re.IGNORECASE)

    print(df1.shape); df1 = df1.drop_duplicates(subset=['cats'], keep='first')
    print(df1.shape); df1 = df1[df1['cats'].str.contains(regex)].reset_index(drop=True)
    print(df1.shape);
    
    df1.to_csv('Categories.csv')
else:
    df1 = pd.read_csv('Categories.csv')
    
df1

(135, 2)
(122, 2)
(16, 2)


,cats,parent
0,Thể_loại:Triều đại Việt Nam,Thể_loại:Triều đại Việt Nam
1,Thể loại:Nhà Đinh,Thể_loại:Triều đại Việt Nam
2,Thể loại:Nhà Hậu Trần,Thể_loại:Triều đại Việt Nam
3,Thể loại:Nhà Hồ,Thể_loại:Triều đại Việt Nam
4,Thể loại:Nhà Lê trung hưng,Thể_loại:Triều đại Việt Nam
5,Thể loại:Nhà Lê sơ,Thể_loại:Triều đại Việt Nam
6,Thể loại:Nhà Tiền Lê,Thể_loại:Triều đại Việt Nam
7,Thể loại:Nhà Lý,Thể_loại:Triều đại Việt Nam
8,Thể loại:Nhà Tiền Lý,Thể_loại:Triều đại Việt Nam
9,Thể loại:Nhà Mạc,Thể_loại:Triều đại Việt Nam


In [45]:
if not os.path.isfile('Pages.csv'):
    all_pages = getAllPages(SESSION, df1.cats)
    
    print(all_pages.shape); allnodup = all_pages.drop_duplicates(subset=['page'], keep='first')
    
    words_place = [
        'Thời kỳ',
        'Nhà',
        'Triều đại',
        'cổ đại'
    ]

    regex = re.compile('|'.join(words_place))
    print(allnodup.shape); allnodup = allnodup[allnodup['page'].str.contains(regex)]
    
    allnodup.to_csv('Pages.csv', index=False)
else:
    allnodup = pd.read_csv('Pages.csv')

allnodup.sort_values('page')

,page,cat
17,Kênh Nhà Lê,Thể loại:Nhà Tiền Lê
5,Nhà Hậu Lê,Thể_loại:Triều đại Việt Nam
1,Nhà Hậu Trần,Thể_loại:Triều đại Việt Nam
6,Nhà Hồ,Thể_loại:Triều đại Việt Nam
2,Nhà Lê sơ,Thể_loại:Triều đại Việt Nam
3,Nhà Lê trung hưng,Thể_loại:Triều đại Việt Nam
7,Nhà Lý,Thể_loại:Triều đại Việt Nam
8,Nhà Mạc,Thể_loại:Triều đại Việt Nam
9,Nhà Nguyễn,Thể_loại:Triều đại Việt Nam
4,Nhà Ngô,Thể_loại:Triều đại Việt Nam


In [41]:
from tqdm import tqdm

In [46]:
tqdm.pandas()
allnodup.loc[:,'infoboxes'] = allnodup['page'].progress_apply(title_to_infoboxes)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.47it/s]


In [47]:
allnodup.to_csv('Pages_Infoboxes.csv')
allnodup

,page,cat,infoboxes
0,Nhà Đinh,Thể_loại:Triều đại Việt Nam,Infobox Former Country\nChú thích web
1,Nhà Hậu Trần,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
2,Nhà Lê sơ,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
3,Nhà Lê trung hưng,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
4,Nhà Ngô,Thể_loại:Triều đại Việt Nam,Infobox former country\n
5,Nhà Hậu Lê,Thể_loại:Triều đại Việt Nam,
6,Nhà Hồ,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
7,Nhà Lý,Thể_loại:Triều đại Việt Nam,
8,Nhà Mạc,Thể_loại:Triều đại Việt Nam,Infobox Former Country\n
9,Nhà Nguyễn,Thể_loại:Triều đại Việt Nam,Infobox former country\n Chú thích\n


In [57]:
# Fetch the HTML content of the webpage
def infobox_dict_thing(url):
    response = requests.get(url)
    html = response.json()['parse']['text']

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Find the infobox template and extract its contents
    infobox = soup.find("table", {"class": "infobox"})
    if not infobox:
        return {}
        
    rows = infobox.find_all("tr")

    # Convert the infobox contents into a Pandas DataFrame
    data = {}
    for row in rows:
        cells = row.find_all(["th", "td"])
        if len(cells) > 1:
            key = cells[0].get_text().strip()
            value = cells[1].get_text().strip()
            data[key] = value

    return data

In [58]:
def iterate_dict_infobox(url, list_of_loc, infoboxer):
    list_dict = []
    maxi = len(list_of_loc)
    i = 0
    for loc in list_of_loc:
        dict2 = {'page': loc, **infoboxer(url + loc)}
        list_dict.append(dict2)
        print(dict2)
        print(f'{i/maxi*100:.2f}%', end='\r')
        i += 1


    return list_dict

In [59]:
# url = 'https://vi.wikipedia.org/w/api.php?action=parse&prop=wikitext&format=json&page='
# %lprun -f iterate_dict_infobox iterate_dict_infobox(url, allnodup.page[0:20], infobox_dict_thing)

HTMLURL = 'https://vi.wikipedia.org/w/api.php?action=parse&prop=text&formatversion=2&format=json&page='

alldata = iterate_dict_infobox(HTMLURL, allnodup.page, infobox_dict_thing)

{'page': 'Nhà Đinh', 'Thủ\xa0đô': 'Hoa Lư', 'Thành\xa0phố\xa0lớn\xa0nhất': 'Đại La', 'Ngôn\xa0ngữ\xa0thông\xa0dụng': 'Hán ngữ trung đại', 'Tôn\xa0giáo\xa0chính': 'Phật giáo, Đạo giáo,...', 'Chính\xa0phủ': 'Quân chủ', 'Hoàng đế': '', '•\xa0968-979': 'Đinh Tiên Hoàng', '•\xa0979-980': 'Đinh Phế Đế', 'Lịch\xa0sử': '', '•\xa0Đinh Bộ Lĩnh đánh bại 12 sứ quân, lên ngôi hoàng đế': '968', '•\xa0Lê Hoàn soán ngôi Nhà Đinh': '980', 'Đơn\xa0vị\xa0tiền\xa0tệ': 'Tiền xu', 'Tiền thân\nKế tục\n\n\n\n\n\n\n\nNhà Ngô\n\n\n\n\n\n\nNhà Tiền Lê': 'Tiền thân', 'Tiền thân': 'Kế tục', 'Nhà Ngô': '', '': 'Nhà Ngô', 'Nhà Tiền Lê': ''}
{'page': 'Nhà Hậu Trần', 'Vị\xa0thế': 'Đế quốc', 'Thủ\xa0đô': 'Mô Độ (1407 - 1409)Bình Than (1409 - 1414)', 'Ngôn\xa0ngữ\xa0thông\xa0dụng': 'Tiếng ViệtTiếng Hán', 'Tôn\xa0giáo\xa0chính': 'Tam giáo quy nguyên', 'Chính\xa0phủ': 'Quân chủ chuyên chế', 'Quân chủ': '', 'Lịch\xa0sử': '', '•\xa0Giản Định Đế lên ngôi tại Mộ Độ': '1407', '•\xa0Trùng Quang Đế bị bắt tại Hóa Châu': '1414', 

In [60]:
df = pd.DataFrame.from_dict(alldata)
df.shape

(20, 226)

In [61]:
counts = df.count().reset_index()
counts.columns = ['col', 'count']
counts = counts.sort_values(by='count',ascending=False)

In [62]:
df = df.reindex(counts['col'], axis=1)
df

col,page,Lịch sử,,Tiền thân,Đơn vị tiền tệ,Thủ đô,Chính phủ,Ngôn ngữ thông dụng,Hoàng đế,Tôn giáo chính,...,"• Đông Kinh thất thủ, Mạc Mậu Hợp bị giết",• Tàn dư họ Mạc chấm dứt,• 1540,Tiền thân\nKế tục\n\n\n\n\n\n\n\nNhà Lê sơ\n\n\n\n\n\n\nNhà Lê trung hưng,Nhà Lê sơ,Nhà Lê trung hưng,• 1802–1820 (đầu),• 1884–1885 (cuối cùng tronggiai đoạn độc lập),• 1926–1945 (cuối cùng),Nhà Hồ
0,Nhà Đinh,,Nhà Ngô,Kế tục,Tiền xu,Hoa Lư,Quân chủ,Hán ngữ trung đại,,"Phật giáo, Đạo giáo,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nhà Hậu Trần,,Kỷ thuộc Minh,Kế tục,Tiền xu,Mô Độ (1407 - 1409)Bình Than (1409 - 1414),Quân chủ chuyên chế,Tiếng ViệtTiếng Hán,NaN,Tam giáo quy nguyên,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nhà Lê sơ,,Bắc thuộc lần 4,Kế tục,Tiền xu,Đông Kinh(東京 1428 - 1527),Quân chủ tuyệt đối,Việt ngữ trung đại,,"Phật giáo, Nho giáo, Tín ngưỡng dân gian",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nhà Lê trung hưng,,Nhà Lê sơ,Kế tục,Tiền xu,Vạn Lại (tạm thời)(1533–1597)Đông Kinh(1597–1789),Chế độ quân chủ,Tiếng Việt trung đại,,Nho giáo Phật giáo Đạo giáo Công giáo Tín ngưỡ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nhà Ngô,,Bắc thuộc lần 3,Kế tục,Tiền xu,Cổ Loa,Quân chủ,Việt ngữ và Hán ngữ trung đại,NaN,"Phật giáo, Đạo giáo, Cảnh giáo[1]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Nhà Hậu Lê,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Nhà Hồ,,Nhà Trần,Kế tục,"Thông bảo hội sao (tiền giấy), Thánh Nguyên th...","Tây Đô20°4′B 105°36′Đ﻿ / ﻿20,067°B 105,6°Đ﻿ / ...",Quân chủ,Tiếng Việt trung đại,,"Phật giáo, Đạo giáo, Tín ngưỡng Việt Nam",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Nhà Lý,,Nhà Tiền Lê,Kế tục,Tiền xu,Hoa Lư (1009 - 1010)Thăng Long (1010 - 1225),Quân chủ,Tiếng Việt trung đại,,Phật giáo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Nhà Mạc,,Nhà Lê sơ,Kế tục,Tiền xu,Đông Kinh,Quân chủ,Tiếng Việt,,Tam giáo quy nguyên,...,1592,1677,2.000.000,Tiền thân,,,NaN,NaN,NaN,NaN
9,Nhà Nguyễn,,Chúa Nguyễn,Kế tục,Quan,"Phú Xuân (nay là Huế)16°28′B 107°36′Đ﻿ / ﻿16,4...",Quân chủ chuyên chế,Tiếng ViệtVăn ngôn Trung Quốc[1][2][Ghi chú 1]...,,"Nho giáo, Phật giáo, Công giáo, Tín ngưỡng dân...",...,NaN,NaN,NaN,NaN,NaN,NaN,Gia Long,Hàm Nghi,Bảo Đại,NaN


In [59]:
df.to_csv('htmlparse.csv', index=False)
df.shape

(986, 277)

In [60]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)